In [1]:
# Import the necessary libraries, modules
import pandas as pd # Pandas library for reading '.csv' files as dataframes
import numpy as np # Numpy library for creating and modifying arrays.
from keras.layers import Dense, LSTM, Embedding # Import layers from Keras
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
raw_data = pd.read_csv('data.csv', encoding='latin-1') # Read the data as a DataFrame using Pandas
#raw_test_data = pd.read_csv('test.csv', encoding='latin-1')

print(raw_data.shape) # Print the dimensions of train DataFrame
print(raw_data.columns) # Print the column names of the DataFrame
print('\n')
print(raw_data.head(5)) # Print the top few records

(57280, 3)
Index(['field', 'categories', 'converse'], dtype='object')


   field     categories                                           converse
0      0   PRESCRIPTION  patients aware that he needs rov for refill na...
1      1   ASK_A_DOCTOR  mom wants to know if the drugname needs some d...
2      2   ASK_A_DOCTOR  patients to discuss drugname she says she has ...
3      3  MISCELLANEOUS  fyi nortryptline medication patient prescripti...
4      4  MISCELLANEOUS  letter of patient establishment request name s...


In [3]:
raw_data.isnull().sum()

field          0
categories     0
converse      36
dtype: int64

In [4]:
raw_data = raw_data.dropna(axis = 0,how = 'any')
#df.dropna(axis=1, how='any')

In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57244 entries, 0 to 57279
Data columns (total 3 columns):
field         57244 non-null int64
categories    57244 non-null object
converse      57244 non-null object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [6]:
# Print the unique classes and their counts/frequencies
classes = np.unique(raw_data['categories'], return_counts=True) # np.unique returns a tuple with class names and counts
print(classes[0]) #Print the list of unique classes
print(classes[1]) #Print the list of frequencies of the above classes

['APPOINTMENTS' 'ASK_A_DOCTOR' 'JUNK' 'LAB' 'MISCELLANEOUS' 'PRESCRIPTION']
[13870 11800    10  4321 12147 15096]


In [18]:

l = 0
m = 0
for i in x_train:
    l = len(i)
    if(l > m):
        m = l;
print(m)


#max([for i in len(x_train)len(xtrain[i])])

646


In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000) #Tokenizer is used to tokenize text
tokenizer.fit_on_texts(raw_data.converse) #Fit this to our corpus

x_train = tokenizer.texts_to_sequences(raw_data.converse) #'text to sequences converts the text to a list of indices
x_train = pad_sequences(x_train, maxlen=650) #pad_sequences makes every sequence a fixed size list by padding with 0s 
#x_test = tokenizer.texts_to_sequences(raw_test_data.converse) 
#x_test = pad_sequences(x_test, maxlen=50)

x_train.shape  # Check the dimensions of x_train and x_test 


(57244, 650)

In [22]:
#x_train[1]

In [23]:
from keras.utils.np_utils import to_categorical # This convers the categories to one-hot vectors(Dummies)

unique_labels = list(raw_data.categories.unique()) 
y_train = np.array([unique_labels.index(i) for i in raw_data.categories]) # Convert the word labels to indeces
y_train = to_categorical(y_train) # Dummify the labels

In [28]:
y_train.shape

(57244, 6)

In [27]:
import keras.backend as K # This 'K' can be used to create user defined functions in keras

# Define a custom function in keras to compute recall.
# Arguments:
# y_true - Actual labels
# y_pred - Predicted labels
def recall(y_true, y_pred):
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    PP = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = TP / (PP + K.epsilon())
    return recall

In [29]:
# Building an LSTM model
model = Sequential() # Call Sequential to initialize a network
model.add(Embedding(input_dim=len(tokenizer.word_index), 
                    input_length=650, 
                    output_dim=100)) # Add an embedding layer which represents each unique token as a vector
model.add(LSTM(10, return_sequences=False)) # Add an LSTM layer
model.add(Dense(6, activation='softmax')) # Add an ouput layer. Since classification, 3 nodes for 3 classes.

In [30]:
# Mention the optimizer, Loss function and metrics to be computed
model.compile(optimizer='adam',  # 'Adam' is a variant of gradient descent technique
              loss='categorical_crossentropy', # categorical_crossentropy for multi-class classification
              metrics=['accuracy', recall]) # These metrics are computed and printed for evaluating

model.fit(x_train, y_train, epochs=1, validation_split=0.25)

Train on 42933 samples, validate on 14311 samples
Epoch 1/1
42933/42933 [==============================] - 571s 13ms/step - loss: 0.9614 - acc: 0.6558 - recall: 0.5201 - val_loss: 0.9797 - val_acc: 0.6684 - val_recall: 0.5825


In [31]:
recall(x_train,y_train)

ValueError: operands could not be broadcast together with shapes (57244,650) (57244,6) 